In [ ]:
# 1.1 extract first sentences of what people are saying after they enter

import glob
from utils import in_blocklist

blocklist = []
blocklist.append('AWAYLEN=307 MAXTARGETS=20 WALLCHOPS WATCH=128 WATCHOPTS=A SILENCE=15 MODES=12 CHANTYPES=# PREFIX')
blocklist.append('- * -')
blocklist.append('XxXChatters.Com')
blocklist.append('XxXChatters')
blocklist.append('This server was created')
blocklist.append('operator(s) online')
blocklist.append('is now your displayed host')
blocklist.append('Caps set:')
blocklist.append('MAXCHANNELS')
blocklist.append('http')
blocklist.append('Meltdown')
blocklist.append('14')

first_sentences = []

outfile = open('xxxchatters_entrance_first_words.txt', 'w')

def find_post(lines_list, a):
    for s in lines_list:
        if s[0] is '[':
            if s.find("<") is not -1:
                time = s[s.find("[")+1:s.find("]")]
                author = s[s.find("<")+1:s.find(">")]
                sentence = s[s.find(">")+2:]
                if a == author and sentence not in first_sentences:
                    first_sentences.append(sentence)
                    out = time + ", " + s
                    outfile.write(out + "\n")
                    print(out)
                    return

for file in glob.glob("xxxchatters_logs/subset/*.log"):
    lines = open(file, 'r', encoding='utf-8').readlines()
    index = 0
    for line in lines:
        if not in_blocklist(blocklist, line.strip()): # blocked?
            if ") Quit (" in line or ") has left " in line:
                time = line[line.find("[")+1:line.find("]")]
                author = line[line.find(" * ")+3:line.find(" (")]
                sentence = line[line.find(">")+2:]
                
                #print("left: " + time + ", " + author)# + ", " + sentence)
            if ") has joined " in line:
                time = line[line.find("[")+1:line.find("]")]
                author = line[line.find(" * ")+3:line.find(" (")]
                #print("joined: " + time + ", " + author)
                find_post(lines[index:], author)
                
        index += 1
        
outfile.close()

In [1]:
# 1.2 look at word frequency

import glob
from utils import in_blocklist

blocklist = []
blocklist.append('AWAYLEN=307 MAXTARGETS=20 WALLCHOPS WATCH=128 WATCHOPTS=A SILENCE=15 MODES=12 CHANTYPES=# PREFIX')
blocklist.append('- * -')
blocklist.append('XxXChatters.Com')
blocklist.append('XxXChatters')
blocklist.append('This server was created')
blocklist.append('operator(s) online')
blocklist.append('is now your displayed host')
blocklist.append('Caps set:')
blocklist.append('MAXCHANNELS')
blocklist.append('http')
blocklist.append('Meltdown')
blocklist.append('14')


full_lines = []

for file in glob.glob("xxxchatters_logs/subset/*.log"):
    lines = open(file, 'r', encoding='utf-8').readlines()
    
    for line in lines:
        if not in_blocklist(blocklist, line.strip()): # blocked?
            if line[0] is '[':
                full_lines.append(line)
            
print(len(full_lines))

5173


In [3]:
authors = {}
posts = set()
all_posts = []

for s in full_lines:
    if s.find("<") is not -1:
        time = s[s.find("[")+1:s.find("]")]
        author = s[s.find("<")+1:s.find(">")]
        sentence = s[s.find(">")+2:]
        all_posts.append(sentence)
        if author not in authors: # save things
            authors[author] = [] # init list for author
        authors[author].append(sentence.strip()) 
    
print('Different Authors: ' + str(len(authors)) + '\n')

Different Authors: 303



In [5]:
for post in all_posts:
    word_list = post.split()
print(all_posts[1])

Greetings to all...

